# Implicit Statistical Reasoning in Transformers

In [ ]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
import logging 
import pandas as pd
import os
import glob
from dataclasses import dataclass, field
import matplotlib.pyplot as plt
from pathlib import Path


# Logging format: save logs to logs/ folder + console output
os.makedirs('logs', exist_ok=True)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('logs/experiment.log'),
        logging.StreamHandler()
    ]
)


# Random seeds
def set_seed(seed: int = 0):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(0)


def set_plotting_style():
    """Sets up standard plotting style."""
    plt.rcParams.update({
        'text.usetex': False,
        'font.family': 'serif',
        'font.size': 12,
        'axes.labelsize': 14,
        'axes.titlesize': 16,
        'xtick.labelsize': 11,
        'ytick.labelsize': 11,
        'legend.fontsize': 11,
        'figure.figsize': (10, 7),
        'axes.grid': True,
        'grid.alpha': 0.3,
        'lines.linewidth': 2.5,
    })
set_plotting_style()


# PyTorch device selection
DEVICE = torch.device(
    'cuda' if torch.cuda.is_available() 
    else 'mps' if torch.backends.mps.is_available() 
    else 'cpu'
)
logging.info(f'Using device: {DEVICE}')

In [ ]:
@dataclass
class DataConfig:
    """Controls the generation of synthetic tasks."""
    d: int = 16  # Input dimension
    n_ctx: int = 32  # Number of context pairs
    sigma_k: float = 3.0  # Task A: Shift magnitude
    sigma_min: float = 0.5  # Task B: Variance lower bound
    sigma_max: float = 3.0  # Task B: Variance upper bound
    train_episodes: int = 50000  # Size of training dataset
    val_episodes: int = 5000  # Size of validation dataset


@dataclass
class TrainConfig:
    """Controls the optimization loop."""
    seeds: list[int] = field(default_factory=lambda: [0, 1, 2])
    batch_size: int = 64
    epochs: int = 20
    lr: float = 3e-4
    device: torch.device = DEVICE


# Initialize Global Configs
data_cfg = DataConfig()
train_cfg = TrainConfig()

## Sampling Tasks

### Dataset Wrapper

In [ ]:
def make_episode(
    sample_task_params_fn: callable,
    sample_data_fn: callable,
    n_ctx: int,
    d: int,
    **task_kwargs
):
    """Generic episode constructor."""
    task_params = sample_task_params_fn(d=d, **task_kwargs)

    x_ctx, y_ctx = sample_data_fn(task_params, n_ctx, d)
    x_q, y_q = sample_data_fn(task_params, 1, d)

    return {
        'context_x': x_ctx,
        'context_y': y_ctx,
        'query_x': x_q[0],
        'query_y': y_q[0],
        'task_params': task_params,
    }


class EpisodeDataset(Dataset):
    """PyTorch Dataset for generating episodes on-the-fly."""
    def __init__(
        self,
        sample_task_params_fn,
        sample_data_fn,
        n_ctx: int,
        d: int,
        num_episodes: int,
        device: torch.device = torch.device('cpu'),
        **task_kwargs
    ):
        self.sample_task_params_fn = sample_task_params_fn
        self.sample_data_fn = sample_data_fn
        self.n_ctx = n_ctx
        self.d = d
        self.num_episodes = num_episodes
        self.task_kwargs = dict(task_kwargs)
        self.device = device

    def __len__(self):
        return self.num_episodes

    def __getitem__(self, idx):
        """
        Each batch from  yields:
        {
        'context_x': (B, n_ctx, d),
        'context_y': (B, n_ctx),
        'query_x':   (B, d),
        'query_y':   (B,),
        }
        """
        episode = make_episode(
            sample_task_params_fn=self.sample_task_params_fn,
            sample_data_fn=self.sample_data_fn,
            n_ctx=self.n_ctx,
            d=self.d,
            **self.task_kwargs
        )

        # Convert to torch tensors where appropriate
        return {
            'context_x': torch.as_tensor(episode['context_x'], dtype=torch.float32, device=self.device),
            'context_y': torch.as_tensor(episode['context_y'], dtype=torch.long, device=self.device),
            'query_x': torch.as_tensor(episode['query_x'], dtype=torch.float32, device=self.device),
            'query_y': torch.as_tensor(episode['query_y'], dtype=torch.float32, device=self.device),
            'task_params': episode['task_params'],  # keep as Python object
        }

### Task A: Shifted Mean Discrimination

In [ ]:
def sample_task_A_params(d: int, sigma_k: float) -> dict:
    """
    Samples task parameters for Task A: Mean Discrimination.

    Arguments:
    d: data dimension
    sigma_k: standard deviation of the shift, k

    Returns (mu, k).
    """
    mu = np.random.randn(d)
    mu /= np.linalg.norm(mu) 

    k = np.random.randn(d) * sigma_k
    return {'mu': mu, 'k': k}


def sample_task_A_data(task_params: dict, n: int, d: int) -> tuple[np.ndarray, np.ndarray]:
    """
    Samples labeled data from mean discrimination task.
    Returns {x, y}_1^n with y in {0,1}.
    """
    mu, k = task_params['mu'], task_params['k']
    y = np.random.randint(0, 2, size=n)
    means = np.where(y[:, None] == 1, mu + k, -mu + k)
    x = means + np.random.randn(n, d)
    return x, y


def task_A_llr(x: np.ndarray, mu: np.ndarray, k: np.ndarray) -> np.ndarray:
    """
    Bayes-optimal log-likelihood ratio for mean discrimination.
    """
    return 2.0 * (x - k) @ mu

### Task B: Variance Discrimination

In [ ]:
def sample_task_B_params(d: int, sigma_min: float = 0.5, sigma_max: float = 3.0) -> dict:
    """
    Samples variances (sigma_0, sigma_1) from uniform distributions.
    """
    sigma_0 = np.random.uniform(sigma_min, sigma_max)
    sigma_1 = np.random.uniform(sigma_min, sigma_max)
    return {'sigma_0': sigma_0, 'sigma_1': sigma_1}


def sample_task_B_data(task_params: dict, n: int, d: int) -> tuple[np.ndarray, np.ndarray]:
    """
    Samples labeled data from variance discrimination task.
    """
    sigma_0 = task_params['sigma_0']
    sigma_1 = task_params['sigma_1']

    y = np.random.randint(0, 2, size=n)
    sigmas = np.where(y == 1, sigma_1, sigma_0)

    x = np.random.randn(n, d) * sigmas[:, None]
    return x, y


def task_B_llr(x: np.ndarray, sigma_0: float, sigma_1: float) -> np.ndarray:
    """
    Bayes-optimal log-likelihood ratio for variance discrimination.
    """
    d = x.shape[1]
    quad = np.sum(x**2, axis=1)

    return (
        0.5 * (1.0 / sigma_0**2 - 1.0 / sigma_1**2) * quad
        + d * np.log(sigma_0 / sigma_1)
    )

### Training and Validation Dataset

In [ ]:
set_seed(0)
train_loader_A_no_nuisance = DataLoader(
    EpisodeDataset(
        sample_task_params_fn=sample_task_A_params,
        sample_data_fn=sample_task_A_data,
        n_ctx=data_cfg.n_ctx,
        d=data_cfg.d,
        num_episodes=data_cfg.train_episodes,
        sigma_k=0.0,  # No nuisance shift
        device=train_cfg.device
    ),
    batch_size=train_cfg.batch_size, shuffle=True, drop_last=True
)
val_loader_A_no_nuisance = DataLoader(
    EpisodeDataset(
        sample_task_params_fn=sample_task_A_params,
        sample_data_fn=sample_task_A_data,
        n_ctx=data_cfg.n_ctx,
        d=data_cfg.d,
        num_episodes=data_cfg.val_episodes,
        sigma_k=0.0,  # No nuisance shift
        device=train_cfg.device
    ),
    batch_size=train_cfg.batch_size, shuffle=False, drop_last=False
)
train_loader_A = DataLoader(
    EpisodeDataset(
        sample_task_params_fn=sample_task_A_params,
        sample_data_fn=sample_task_A_data,
        n_ctx=data_cfg.n_ctx,
        d=data_cfg.d,
        num_episodes=data_cfg.train_episodes,
        sigma_k=data_cfg.sigma_k,
        device=train_cfg.device
    ),
    batch_size=train_cfg.batch_size, shuffle=True, drop_last=True
)
val_loader_A = DataLoader(
    EpisodeDataset(
        sample_task_params_fn=sample_task_A_params,
        sample_data_fn=sample_task_A_data,
        n_ctx=data_cfg.n_ctx,
        d=data_cfg.d,
        num_episodes=data_cfg.val_episodes,
        sigma_k=data_cfg.sigma_k,
        device=train_cfg.device
    ),
    batch_size=train_cfg.batch_size, shuffle=False, drop_last=False
)
val_loader_A_OOD = DataLoader(
    EpisodeDataset(
        sample_task_params_fn=sample_task_A_params,
        sample_data_fn=sample_task_A_data,
        n_ctx=data_cfg.n_ctx,
        d=data_cfg.d,
        num_episodes=data_cfg.val_episodes,
        sigma_k=3*data_cfg.sigma_k,  # OOD nuisance variables
        device=train_cfg.device
    ),
    batch_size=train_cfg.batch_size, shuffle=False, drop_last=False
)
train_loader_B = DataLoader(
    EpisodeDataset(
        sample_task_params_fn=sample_task_B_params,
        sample_data_fn=sample_task_B_data,
        n_ctx=data_cfg.n_ctx,
        d=data_cfg.d,
        num_episodes=data_cfg.train_episodes,
        sigma_min=data_cfg.sigma_min,
        sigma_max=data_cfg.sigma_max,
        device=train_cfg.device
    ),
    batch_size=train_cfg.batch_size, shuffle=True, drop_last=True
)
val_loader_B = DataLoader(
    EpisodeDataset(
        sample_task_params_fn=sample_task_B_params,
        sample_data_fn=sample_task_B_data,
        n_ctx=data_cfg.n_ctx,
        d=data_cfg.d,
        num_episodes=data_cfg.val_episodes,
        sigma_min=data_cfg.sigma_min,
        sigma_max=data_cfg.sigma_max,
        device=train_cfg.device
    ),
    batch_size=train_cfg.batch_size, shuffle=False, drop_last=False
)

## Training and Evaluation

In [ ]:
def train_step(model: nn.Module, batch: dict, optimizer: optim.Optimizer, loss_fn: nn.Module, scheduler: optim.lr_scheduler._LRScheduler = None):
    model.train()
    optimizer.zero_grad()

    logits = model(
        context_x=batch['context_x'],
        context_y=batch['context_y'],
        query_x=batch['query_x'],
    )

    target = batch['query_y'].float().view(-1, 1) 
    loss = loss_fn(logits, target)
    
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
    optimizer.step()
    if scheduler is not None:
        scheduler.step()
    
    with torch.no_grad():
        probs = torch.sigmoid(logits)
        preds = (probs > 0.5).float()
        acc = (preds == target).float().mean().item()

    return loss.item(), acc


@torch.no_grad()
def eval_step(model: nn.Module, batch: dict, loss_fn: nn.Module):
    model.eval()

    logits = model(
        context_x=batch['context_x'],
        context_y=batch['context_y'],
        query_x=batch['query_x'],
    )

    target = batch['query_y'].float().view(-1, 1)
    loss = loss_fn(logits, target)
    
    probs = torch.sigmoid(logits)
    preds = (probs > 0.5).float()
    acc = (preds == target).float().mean().item()

    return loss.item(), acc, logits.cpu(), target.cpu()


def run_epoch(
        model: nn.Module, dataloader: DataLoader, optimizer: optim.Optimizer = None, scheduler: optim.lr_scheduler._LRScheduler = None
    ) -> dict:
    is_train = optimizer is not None
    loss_fn = nn.BCEWithLogitsLoss()

    total_loss = 0.0
    correct = 0
    total = 0
    all_logits, all_labels = [], []

    for batch in dataloader:
        if is_train:
            loss, acc = train_step(model, batch, optimizer, loss_fn, scheduler)
            logits = None
            labels = None
        else:
            loss, acc, logits, labels = eval_step(model, batch, loss_fn)
            all_logits.append(logits)
            all_labels.append(labels)

        B = batch['query_y'].numel()
        total_loss += loss * B
        correct += int(acc * B)
        total += B

    metrics = {
        'loss': total_loss / total,
        'acc': correct / total,
    }

    if not is_train:
        metrics['logits'] = torch.cat(all_logits)
        metrics['labels'] = torch.cat(all_labels)

    return metrics


def train_model(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    epochs: int = 10,
    lr: float = 1e-3,
    save_best: bool = False,
) -> tuple[list[dict], dict | None]:
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer, 
        max_lr=lr, 
        epochs=epochs, 
        steps_per_epoch=len(train_loader),
        pct_start=0.3,  # Warmup for 30% of time
    )

    history = []
    best_val_loss = float('inf')
    best_state = None

    for epoch in range(epochs):
        train_metrics = run_epoch(model, train_loader, optimizer, scheduler)
        val_metrics = run_epoch(model, val_loader)

        # Store metrics for this epoch
        history.append({
            'epoch': epoch,
            'train_loss': train_metrics['loss'],
            'train_acc': train_metrics['acc'],
            'val_loss': val_metrics['loss'],
            'val_acc': val_metrics['acc']
        })

        if save_best and val_metrics['loss'] < best_val_loss:
            best_val_loss = val_metrics['loss']
            best_state = {
                k: v.detach().cpu()
                for k, v in model.state_dict().items()
            }

        logging.info(
            f"[Epoch {epoch:03d}] "
            f"Train loss={train_metrics['loss']:.4f}, acc={train_metrics['acc']:.3f} | "
            f"Val loss={val_metrics['loss']:.4f}, acc={val_metrics['acc']:.3f}"
        )

    return history, best_state


def run_multiseed_experiment(
    model_class: type[nn.Module],
    model_kwargs: dict,
    train_loader: DataLoader,
    val_loader: DataLoader,
    seeds: list[int] | None,
    epochs: int = 10,
    lr: float = 1e-3,
    device: torch.device = torch.device('cpu'),
    experiment_name: str = 'task',
    checkpoint_dir: str | Path = 'checkpoints',
    data_output_dir: str | Path = 'results',
) -> pd.DataFrame:
    if seeds is None:
        seeds = [0, 1, 2]

    checkpoint_dir = Path(checkpoint_dir)
    data_output_dir = Path(data_output_dir)

    all_results = []
    model_name = model_class.__name__

    logging.info(f'Running experiment for model: {model_name} with seeds: {seeds}')
    
    for seed in seeds:
        logging.info(f'Starting training with seed: {seed}')
        set_seed(seed)

        # Re-initialize Model Fresh 
        model = model_class(**model_kwargs).to(device)
        history, best_state = train_model(
            model,
            train_loader,
            val_loader,
            epochs=epochs,
            lr=lr,
            save_best=True,
        )

        # Save best model checkpoint
        ckpt_path = checkpoint_dir / experiment_name / f'{model_name}_seed{seed}.pt'
        os.makedirs(ckpt_path.parent, exist_ok=True)
        torch.save({
            'model_class': model_name,
            'model_kwargs': model_kwargs,
            'seed': seed,
            'best_state_dict': best_state,
        }, ckpt_path)
        logging.info(f'Saved checkpoint to {ckpt_path}')

        # Add metadata and store
        for epoch_data in history:
            epoch_data.update({
                'seed': seed,
                'model': model_name,
                'experiment': experiment_name,
            })
            all_results.append(epoch_data)

    # Restore global seed
    set_seed(0) 

    df = pd.DataFrame(all_results)
    # Save results to CSV
    data_output_dir = Path(data_output_dir)
    data_output_dir.mkdir(parents=True, exist_ok=True)
    csv_path = data_output_dir / experiment_name / f'{model_name}_results.csv'
    os.makedirs(csv_path.parent, exist_ok=True)
    df.to_csv(csv_path, index=False)
    logging.info(f'Saved results to {csv_path}')
    return df 

## Main Model Architecture

In [ ]:
class ICLTransformer(nn.Module):
    def __init__(self, d_in, n_ctx, d_model=128, n_layers=2, n_heads=4):
        super().__init__()
        self.d_model = d_model
        self.n_ctx = n_ctx
        # Embeddings
        self.x_proj = nn.Linear(d_in, d_model)
        self.y_proj = nn.Linear(1, d_model)
        self.query_proj = nn.Linear(d_in, d_model)
        self.pos_emb = nn.Parameter(torch.randn(1, n_ctx + 1, d_model) * 0.02)
        layer = nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=n_heads, 
            dim_feedforward=4*d_model, 
            dropout=0.0, # No dropout for synthetic tasks required
            batch_first=True,
            activation='gelu',
            norm_first=False,
        )
        self.transformer = nn.TransformerEncoder(layer, num_layers=n_layers)
        self.head = nn.Linear(d_model, 1)

    def forward(self, context_x, context_y, query_x):
        B = context_x.shape[0]
        N = context_x.shape[1]
        # Embed Context: Combine x and y
        ctx_emb = self.x_proj(context_x) + self.y_proj(context_y.unsqueeze(-1).float())
        # Embed Query: It has no y, so we just embed x
        q_emb = self.query_proj(query_x).unsqueeze(1) # [B, 1, D]
        # Concatenate: [Ctx_1, ..., Ctx_N, Query]
        seq = torch.cat([ctx_emb, q_emb], dim=1) # [B, N+1, D]
        # Add Positional Embeddings
        seq = seq + self.pos_emb[:, :seq.shape[1], :]
        # Transformer Pass
        out = self.transformer(seq)
        # Predict only on the Query token (the last one)
        query_out = out[:, -1, :]
        logits = self.head(query_out)
        return logits

## PART I: Recovery of Optimal Tests

In [ ]:
# Experiment: Task A (In sample validation)
run_multiseed_experiment(
    model_class=ICLTransformer,
    model_kwargs={
        'd_in': data_cfg.d,
        'n_ctx': data_cfg.n_ctx,
        'd_model': 128,
        'n_layers': 2,
        'n_heads': 4,
    },
    train_loader=train_loader_A,
    val_loader=val_loader_A,
    seeds=train_cfg.seeds,
    epochs=train_cfg.epochs,
    lr=train_cfg.lr,
    device=train_cfg.device,
    experiment_name='task_A_regular',
    checkpoint_dir='checkpoints',
    data_output_dir='results',
)

# Experiment: Task A (OOD validation)
run_multiseed_experiment(
    model_class=ICLTransformer,
    model_kwargs={
        'd_in': data_cfg.d,
        'n_ctx': data_cfg.n_ctx,
        'd_model': 128,
        'n_layers': 2,
        'n_heads': 4,
    },
    train_loader=train_loader_A,
    val_loader=val_loader_A_OOD,
    seeds=train_cfg.seeds,
    epochs=train_cfg.epochs,
    lr=train_cfg.lr,
    device=train_cfg.device,
    experiment_name='task_A_OOD',
    checkpoint_dir='checkpoints',
    data_output_dir='results',
)

# Experiment: Task B
run_multiseed_experiment(
    model_class=ICLTransformer,
    model_kwargs={
        'd_in': data_cfg.d,
        'n_ctx': data_cfg.n_ctx,
        'd_model': 128,
        'n_layers': 2,
        'n_heads': 4,
    },
    train_loader=train_loader_B,
    val_loader=val_loader_B,
    seeds=train_cfg.seeds,
    epochs=train_cfg.epochs,
    lr=train_cfg.lr,
    device=train_cfg.device,
    experiment_name='task_B_regular',
    checkpoint_dir='checkpoints',
    data_output_dir='results',
)

## PART II: Failure Modalities

### Model Ablation Architectures

In [ ]:
class ICLTransformerInterleavedEmbeddings(nn.Module):
    """ 
    Context provided as (x, y) pairs interleaved in sequence.
    Tests the inductive bias of using (x, y) together, which is broken by interleaving.
    """
    def __init__(self, d_in, n_ctx, d_model=128, n_layers=2, n_heads=4, max_len=512):
        assert max_len >= 2 * n_ctx + 1, 'max_len must be at least 2*n_ctx + 1'
        super().__init__()
        self.d_model = d_model
        
        # Embeddings
        # Projects input data x -> d_model
        self.x_embed = nn.Linear(d_in, d_model)
        # Embeds binary labels y -> d_model
        self.y_embed = nn.Embedding(2, d_model)
        # Learned positional embedding
        self.pos_embed = nn.Parameter(torch.randn(1, max_len, d_model) * 0.02)
        
        # Transformer Encoder (GPT-style)
        layer = nn.TransformerEncoderLayer(
            d_model=d_model, 
            nhead=n_heads, 
            dim_feedforward=4*d_model, 
            dropout=0.0, # No dropout for synthetic tasks required
            batch_first=True,
            activation='gelu',
            norm_first=False,
        )
        self.transformer = nn.TransformerEncoder(layer, num_layers=n_layers)
        self.head = nn.Linear(d_model, 1)

    def forward(self, context_x, context_y, query_x):
        """
        Input:
            context_x: (B, N, d)
            context_y: (B, N)
            query_x:   (B, d)
        """
        B, N, _ = context_x.shape
        device = context_x.device
        # Embed inputs 
        ctx_x_emb = self.x_embed(context_x)  # (B, N, d_model)
        qry_x_emb = self.x_embed(query_x.unsqueeze(1))  # (B, 1, d_model)
        ctx_y_emb = self.y_embed(context_y)  # (B, N, d_model)
        # Interleave Sequence 
        # Sequence: [x1, y1, x2, y2, ..., xN, yN, x_query]
        # Total length = 2*N + 1
        seq_len = 2*N + 1
        seq_emb = torch.zeros(B, seq_len, self.d_model, device=device)
        # Evens are X, Odds are Y
        seq_emb[:, 0:2*N:2, :] = ctx_x_emb
        seq_emb[:, 1:2*N:2, :] = ctx_y_emb
        # Last token is Query X
        seq_emb[:, -1, :] = qry_x_emb.squeeze(1)
        # Add Position & Forward
        seq_emb = seq_emb + self.pos_embed[:, :seq_len, :]
        out = self.transformer(seq_emb)
        # Predict on last token (repr of x_query)
        last_token = out[:, -1, :]
        return self.head(last_token)
    

class ICLTransformerNoLabels(nn.Module):
    """
    Context provided, but labels y are removed.
    Tests whether performance relies on (x, y) pairing.
    """
    def __init__(self, d_in, n_ctx, d_model=128, n_layers=2, n_heads=4):
        super().__init__()
        self.x_proj = nn.Linear(d_in, d_model)
        self.query_proj = nn.Linear(d_in, d_model)
        self.pos_emb = nn.Parameter(torch.randn(1, n_ctx + 1, d_model) * 0.02)

        layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=n_heads,
            dim_feedforward=4*d_model,
            dropout=0.0,
            batch_first=True,
            activation='gelu',
        )
        self.transformer = nn.TransformerEncoder(layer, num_layers=n_layers)
        self.head = nn.Linear(d_model, 1)

    def forward(self, context_x, context_y, query_x):
        ctx_emb = self.x_proj(context_x)
        q_emb = self.query_proj(query_x).unsqueeze(1)
        seq = torch.cat([ctx_emb, q_emb], dim=1)
        seq = seq + self.pos_emb[:, :seq.shape[1], :]
        out = self.transformer(seq)
        return self.head(out[:, -1])


class ICLTransformerShuffledLabels(ICLTransformer):
    """
    Labels y are randomly permuted within each batch.
    Preserves label distribution but breaks x-y association.
    """
    def forward(self, context_x, context_y, query_x):
        B, N = context_y.shape
        perm = torch.randperm(N, device=context_y.device)
        shuffled_y = context_y[:, perm]
        return super().forward(context_x, shuffled_y, query_x)


class ICLTransformerShuffledContext(ICLTransformer):
    """
    Context (x, y) pairs are randomly permuted within each batch.
    Tests whether the model relies on the order of context points.
    """
    def forward(self, context_x, context_y, query_x):
        B, N, _ = context_x.shape
        perm = torch.randperm(N, device=context_x.device)
        return super().forward(
            context_x[:, perm],
            context_y[:, perm],
            query_x
        )
    

class ICLTransformerFrozenAttention(ICLTransformer):
    """
    Attention weights are frozen at initialization.
    Only value projections + MLPs train.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        for layer in self.transformer.layers:
            attn = layer.self_attn

            # Freeze Q, K, V projections
            for p in attn.in_proj_weight, attn.in_proj_bias:
                if p is not None:
                    p.requires_grad = False

            # Freeze output projection
            attn.out_proj.weight.requires_grad = False
            attn.out_proj.bias.requires_grad = False


class ICLTransformerFrozenQK(ICLTransformer):
    """
    Query and key projections frozen; values trainable.
    Tests whether matching is essential or only aggregation.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        for layer in self.transformer.layers:
            attn = layer.self_attn
            d = attn.embed_dim
            
            # We define a hook that zeroes out the gradients for the first 2*d rows
            # (which correspond to Q and K in the fused in_proj_weight)
            def get_qk_freeze_hook(d_dim):
                def hook(grad):
                    # grad shape is (3*d, d) for weight, (3*d) for bias
                    # We clone to ensure we don't modify the gradient buffer in place unexpectedly
                    new_grad = grad.clone()
                    # Zero out Q and K parts
                    new_grad[:2*d_dim] = 0.0
                    return new_grad
                return hook

            # Register the hook on the parameters
            attn.in_proj_weight.register_hook(get_qk_freeze_hook(d))
            
            if attn.in_proj_bias is not None:
                attn.in_proj_bias.register_hook(get_qk_freeze_hook(d))


class ICLTransformerFrozenPos(ICLTransformer):
    """
    Positional embeddings are frozen at initialization.
    Tests reliance on learned absolute position.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.pos_emb.requires_grad = False


class ICLTransformerNoPos(ICLTransformer):
    """
    No positional information at all.
    Tests permutation-invariant aggregation.
    """
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        with torch.no_grad():
            self.pos_emb.zero_()
        self.pos_emb.requires_grad = False


class ICLTransformerNoisyLabels(ICLTransformer):
    """
    Injects random label noise during forward pass.
    Tests robustness of evidence aggregation.
    """
    def __init__(self, *args, noise_p=0.2, **kwargs):
        super().__init__(*args, **kwargs)
        self.noise_p = noise_p

    def forward(self, context_x, context_y, query_x):
        if self.training and self.noise_p > 0:
            noise = torch.rand_like(context_y.float()) < self.noise_p
            context_y = context_y.clone()
            context_y[noise] = 1 - context_y[noise]
        return super().forward(context_x, context_y, query_x)

### Ablation Runs on Task A

In [ ]:
ABLATIONS_TASK_A = [
    ('interleaved', ICLTransformerInterleavedEmbeddings, {}),
    ('no_labels', ICLTransformerNoLabels, {}),
    ('shuffled_labels', ICLTransformerShuffledLabels, {}),
    ('shuffled_context', ICLTransformerShuffledContext, {}),
    ('frozen_attention', ICLTransformerFrozenAttention, {}),
    ('frozen_qk', ICLTransformerFrozenQK, {}),
    ('frozen_pos', ICLTransformerFrozenPos, {}),
    ('no_pos', ICLTransformerNoPos, {}),
]
for ablation_name, model_class, model_kwargs in ABLATIONS_TASK_A:
    run_multiseed_experiment(
        model_class=model_class,
        model_kwargs={
            'd_in': data_cfg.d,
            'n_ctx': data_cfg.n_ctx,
            'd_model': 128,
            'n_layers': 2,
            'n_heads': 4,
            **model_kwargs,
        },
        train_loader=train_loader_A,
        val_loader=val_loader_A,
        seeds=train_cfg.seeds,
        epochs=train_cfg.epochs,
        lr=train_cfg.lr,
        device=train_cfg.device,
        experiment_name=f'task_A_ablation_{ablation_name}',
        checkpoint_dir='checkpoints',
        data_output_dir='results',
    )


run_multiseed_experiment(
    model_class=ICLTransformer,
    model_kwargs={
        'd_in': data_cfg.d,
        'n_ctx': 2 * data_cfg.n_ctx,
        'd_model': 128,
        'n_layers': 2,
        'n_heads': 4,
    },
    train_loader=train_loader_A,
    val_loader=val_loader_A,
    seeds=train_cfg.seeds,
    epochs=train_cfg.epochs,
    lr=train_cfg.lr,
    device=train_cfg.device,
    experiment_name='task_A_ablation_context_increase',
    checkpoint_dir='checkpoints',
    data_output_dir='results',
)


NOISE_LEVELS = [0.1, 0.2, 0.4]
for p in NOISE_LEVELS:
    run_multiseed_experiment(
        model_class=ICLTransformerNoisyLabels,
        model_kwargs={
            'd_in': data_cfg.d,
            'n_ctx': data_cfg.n_ctx,
            'd_model': 128,
            'n_layers': 2,
            'n_heads': 4,
            'noise_p': p,
        },
        train_loader=train_loader_A,
        val_loader=val_loader_A,
        seeds=train_cfg.seeds,
        epochs=train_cfg.epochs,
        lr=train_cfg.lr,
        device=train_cfg.device,
        experiment_name=f'task_A_ablation_noisy_labels_p{p}',
        checkpoint_dir='checkpoints',
        data_output_dir='results',
    )

## Statistics

In [ ]:
# -----------------------------
# 1) Load all result CSVs
# -----------------------------
def load_all_results_csvs(results_root="results"):
    paths = sorted(glob.glob(os.path.join(results_root, "**", "*.csv"), recursive=True))
    dfs = []
    for p in paths:
        try:
            df = pd.read_csv(p)
        except Exception as e:
            print(f"[WARN] failed to read {p}: {e}")
            continue

        df["source_path"] = p

        if "experiment" not in df.columns:
            df["experiment"] = os.path.basename(os.path.dirname(p))

        if "seed" in df.columns:
            df["seed"] = df["seed"].astype(int)
        if "epoch" in df.columns:
            df["epoch"] = df["epoch"].astype(int)

        dfs.append(df)

    if not dfs:
        raise RuntimeError(f"No CSVs found under: {results_root}")

    return pd.concat(dfs, ignore_index=True)


# -----------------------------
# 2) Mean ± 95% CI helper
# -----------------------------
def mean_ci95_halfwidth(x: pd.Series):
    x = x.dropna().to_numpy(dtype=float)
    n = len(x)
    if n == 0:
        return np.nan, np.nan, 0
    mean = float(np.mean(x))
    if n == 1:
        return mean, np.nan, 1
    sd = float(np.std(x, ddof=1))
    half = 1.96 * sd / math.sqrt(n)
    return mean, half, n


# -----------------------------
# 3) Final accuracy table (train + val, no halfwidth columns)
# -----------------------------
def final_train_val_table(df_all: pd.DataFrame):
    required = {"experiment", "model", "seed", "epoch", "train_acc", "val_acc"}
    missing = required - set(df_all.columns)
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    last_rows = (
        df_all.sort_values(["experiment", "model", "seed", "epoch"])
              .groupby(["experiment", "model", "seed"], as_index=False)
              .tail(1)
    )

    out = []
    for (exp, model), g in last_rows.groupby(["experiment", "model"]):
        tr_m, tr_h, n1 = mean_ci95_halfwidth(g["train_acc"])
        va_m, va_h, n2 = mean_ci95_halfwidth(g["val_acc"])
        n = min(n1, n2)

        out.append({
            "experiment": exp,
            "model": model,
            "n_seeds": n,
            "train_acc_ci95": (f"{tr_m:.4f} ± {tr_h:.4f}" if not np.isnan(tr_h) else f"{tr_m:.4f}"),
            "val_acc_ci95":   (f"{va_m:.4f} ± {va_h:.4f}" if not np.isnan(va_h) else f"{va_m:.4f}"),
        })

    return pd.DataFrame(out).sort_values(["experiment", "model"]).reset_index(drop=True)


# -----------------------------
# 4) Plot train/val curves with 95% CI for one task
# -----------------------------
def plot_learning_curves_ci95(
    df_all: pd.DataFrame,
    experiment: str,
    model: str | None = None,
    out_path: str | None = None,
):
    required = {"experiment", "epoch", "seed", "train_acc", "val_acc"}
    missing = required - set(df_all.columns)
    if missing:
        raise ValueError(f"Missing required columns: {missing}")

    df = df_all[df_all["experiment"] == experiment].copy()
    if model is not None:
        if "model" not in df.columns:
            raise ValueError("model column missing; cannot filter by model.")
        df = df[df["model"] == model].copy()

    if df.empty:
        raise ValueError(f"No rows found for experiment={experiment} model={model}")

    def agg_mean_ci(x):
        x = x.dropna().to_numpy(dtype=float)
        n = len(x)
        m = float(np.mean(x)) if n else np.nan
        if n <= 1:
            return pd.Series({"mean": m, "ci": np.nan})
        sd = float(np.std(x, ddof=1))
        ci = 1.96 * sd / math.sqrt(n)
        return pd.Series({"mean": m, "ci": ci})

    train_stats = df.groupby("epoch")["train_acc"].apply(agg_mean_ci).reset_index()
    val_stats   = df.groupby("epoch")["val_acc"].apply(agg_mean_ci).reset_index()

    # flatten output (robust across pandas versions)
    train_stats = train_stats.pivot(index="epoch", columns="level_1", values="train_acc").reset_index()
    val_stats   = val_stats.pivot(index="epoch", columns="level_1", values="val_acc").reset_index()

    plt.figure()
    plt.plot(train_stats["epoch"], train_stats["mean"], label="train acc (mean)")
    plt.fill_between(
        train_stats["epoch"],
        train_stats["mean"] - train_stats["ci"].fillna(0),
        train_stats["mean"] + train_stats["ci"].fillna(0),
        alpha=0.2,
    )

    plt.plot(val_stats["epoch"], val_stats["mean"], label="val acc (mean)")
    plt.fill_between(
        val_stats["epoch"],
        val_stats["mean"] - val_stats["ci"].fillna(0),
        val_stats["mean"] + val_stats["ci"].fillna(0),
        alpha=0.2,
    )

    title = f"{experiment}" + (f" | {model}" if model is not None else "")
    plt.title(title)
    plt.xlabel("epoch")
    plt.ylabel("accuracy")
    plt.ylim(0.0, 1.0)
    plt.legend()

    if out_path is not None:
        os.makedirs(os.path.dirname(out_path), exist_ok=True)
        plt.savefig(out_path, dpi=200, bbox_inches="tight")
        plt.close()
        print(f"Saved plot to: {out_path}")
    else:
        plt.show()


# 5) Run analysis
df_all = load_all_results_csvs("results")

summary = final_train_val_table(df_all)
print(summary.to_string(index=False))

os.makedirs("analysis_out", exist_ok=True)
summary.to_csv("analysis_out/final_train_val_acc_ci95.csv", index=False)
print("Saved summary to analysis_out/final_train_val_acc_ci95.csv")

experiment_to_plot = "task_A_regular"
model_to_plot = "ICLTransformer"

plot_learning_curves_ci95(
    df_all,
    experiment=experiment_to_plot,
    model=model_to_plot,
    out_path="analysis_out/task_A_regular_learning_curve_ci95.png",
)


In [ ]:
# Plot all experiments and models with available data
experiment_to_plot = "task_A_regular"
model_to_plot = "ICLTransformer"

experiments = df_all["experiment"].unique()
models = df_all["model"].unique()
for exp in experiments:
    for mod in models:
        try:
            plot_learning_curves_ci95(
                df_all,
                experiment=exp,
                model=mod,
                out_path=f"analysis_out/{exp}_{mod}_learning_curve_ci95.png",
            )
        except ValueError:
            # Skip if no data for this combination
            continue


In [ ]:
def load_model_from_checkpoint(ckpt_path, device='cpu'):
    # 1. Load the file
    checkpoint = torch.load(ckpt_path, map_location=device)
    
    # 2. Get the class name and arguments
    class_name = checkpoint['model_class']
    kwargs = checkpoint['model_kwargs']
    state_dict = checkpoint['best_state_dict']
    
    print(f"Loading {class_name} from seed {checkpoint['seed']} from {ckpt_path}")
    
    # 3. Map string name to actual Class object
    # (Add any new ablation classes to this map if needed)
    MODEL_MAP = {
        'ICLTransformer': ICLTransformer,
        'ICLTransformerNoLabels': ICLTransformerNoLabels,
        'ICLTransformerFrozenPos': ICLTransformerFrozenPos,
        'ICLTransformerNoPos': ICLTransformerNoPos,
        'ICLTransformerFrozenQK': ICLTransformerFrozenQK,
        'ICLTransformerShuffledLabels': ICLTransformerShuffledLabels,
        'ICLTransformerShuffledContext': ICLTransformerShuffledContext,
        'ICLTransformerNoisyLabels': ICLTransformerNoisyLabels,
    }
    
    if class_name not in MODEL_MAP:
        raise ValueError(f"Unknown model class: {class_name}")
    
    ModelClass = MODEL_MAP[class_name]
    
    # 4. Re-instantiate the model
    model = ModelClass(**kwargs)
    
    # 5. Load weights
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval() # Set to eval mode by default
    
    return model

path = "checkpoints/task_A_regular/ICLTransformer_seed0.pt"
model = load_model_from_checkpoint(path, device=train_cfg.device)